# Forecasting

Using the data from the region_margins, follow a pycaret  time-series experiment for Margin Forecasting.

In [0]:
!pip install pycaret[full]
dbutils.library.restartPython()

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Requirement a

*** WARNING: max output size exceeded, skipping output. ***

t[full]) (1.26.14)
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


# Load region_margin data

In [0]:
%sql

REFRESH TABLE region_margin;
select * from region_margin;

region,product_line,month,cost_m_usd,revenue_m_usd,margin
us-west1,sql,2023-01-01,0.8,0.6,-0.25000000000000006
us-west1,sql,2023-02-01,0.8,1.0,0.24999999999999994
us-west1,sql,2023-03-01,0.8,1.2,0.4999999999999999
us-west1,sql,2023-04-01,0.8,1.3,0.625
us-west1,sql,2023-05-01,0.8,1.3,0.625
us-west1,sql,2023-06-01,0.8,1.3,0.625
us-west1,sql,2023-07-01,0.8,1.4,0.7499999999999998
us-west1,sql,2023-08-01,0.8,1.5,0.8749999999999999
us-west1,jobs,2023-01-01,1.1,1.1,0.0
us-west1,jobs,2023-02-01,1.1,1.5,0.36363636363636354


# Load data to Dataframe

In [0]:
import pandas as pd

region_margins_sqldf=_sqldf
region_margins_df = region_margins_sqldf.select('margin','product_line','region','month').toPandas()
region_margins_df['month'] = pd.to_datetime(region_margins_df['month'])

display(region_margins_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   margin        32 non-null     float64       
 1   product_line  32 non-null     object        
 2   region        32 non-null     object        
 3   month         32 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 1.1+ KB


# TS Forecasting Experiment - Setup

In [0]:
from pycaret.time_series import *
exp = TSForecastingExperiment()
data = region_margins_df[['month','margin']]
s = exp.setup(data=data,target='margin', fh=3, session_id=1)  


Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7f64dafd01f0>
Traceback (most recent call last):
  File "/databricks/python/lib/python3.10/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/databricks/python/lib/python3.10/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/databricks/python/lib/python3.10/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/databricks/python/lib/python3.10/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'


,Description,Value
0,session_id,1
1,Target,margin
2,Approach,Univariate
3,Exogenous Variables,Present
4,Original data shape,"(32, 2)"
5,Transformed data shape,"(32, 2)"
6,Transformed train set shape,"(29, 2)"
7,Transformed test set shape,"(3, 2)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


# Check Data Stats

In [0]:
s.check_stats()

,Test,Test Name,Data,Property,Setting,Value
0,Summary,Statistics,Transformed,Length,,32.0
1,Summary,Statistics,Transformed,# Missing Values,,0.0
2,Summary,Statistics,Transformed,Mean,,0.303355
3,Summary,Statistics,Transformed,Median,,0.363636
4,Summary,Statistics,Transformed,Standard Deviation,,0.27492
5,Summary,Statistics,Transformed,Variance,,0.075581
6,Summary,Statistics,Transformed,Kurtosis,,-0.617765
7,Summary,Statistics,Transformed,Skewness,,-0.184213
8,Summary,Statistics,Transformed,# Distinct Values,,10.0
9,White Noise,Ljung-Box,Transformed,Test Statictic,"{'alpha': 0.05, 'K': 24}",69.141186


# Compare Experiment Models

In [0]:
best = exp.compare_models()


,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
naive,Naive Forecaster,0.8836,0.5355,0.1175,0.1389,1.000800e+14,1.1111,-3.333000e-01,2.1400
grand_means,Grand Means Forecaster,1.6013,0.8441,0.2209,0.2235,6.078904e+14,1.1336,-2.569179e+30,0.0533
croston,Croston,1.7290,0.9283,0.2377,0.2449,6.233455e+14,1.1891,-5.222455e+30,0.0400



# Plot forecast for 12 months in future

In [0]:
exp.plot_model(best, plot = 'forecast', data_kwargs = {'fh' : 12})


# Residuals

In [0]:
# residuals plot
exp.plot_model(best, plot = 'residuals')



# Prediction

In [0]:
holdout_pred = exp.predict_model(best)


,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,Naive Forecaster,0.0002,0.0001,0.0,0.0,0.0001,0.0001,-2.649131e+23


In [0]:
exp.predict_model(best, fh = 5)


,y_pred
29,0.4286
30,0.4286
31,0.4286
32,0.4286
33,0.4286
